# FINN - Transformation passes
--------------------------------------
* <font size="3">changes (transforms) the given graph</font>
* <font size="3">input: ModelWrapper</font>
* <font size="3">returns the changed model and flag `model_was_changed`</font>